# MindCV简介

MindCV是一个基于MindSpore的计算机视觉库，集成了许多经典的或者最前沿的模型，例如ResNet和SwinTransformer；支持一些最新的数据增强方法，例如AutoAugment；也支持一些主流的图像数据集，例如CIFAR10。

这个实战例子是在CIFAR-10数据集上训练Vision Transformer(ViT)。通过这个例子，您可以了解MindCV训练的基本流程和调试的方法。

## 安装

首先是安装硬件设备的驱动， 详情请见[GPU设备安装参考](https://www.nvidia.cn/geforce/drivers/)或[昇腾设备安装参考](https://support.huawei.com/enterprise/zh/doc/EDOC1100289994)。

然后是安装MindSpore和MindCV。您可以通过学习[MindSpore官网安装教程](https://www.mindspore.cn/install)安装MindSpore最新的版本。安装完成后，可以运行:

```
>>> import mindspore
>>> mindspore.run_check()
```
如果出现mindspore的版本号，则说明安装成功。


MindCV可以通过`pip`安装：
```
pip install mindcv
```

也可以通过源代码安装（推荐用这种方式，能够安装最新的版本）：
```
pip install git+https://github.com/mindspore-lab/mindcv.git
```

注意当前MindCV要求的MindSpore最低版本为`1.8.1`。
如果`import mindcv`没有出现报错，说明MindCV安装成功。


## 关于MindSpore的动态图和静态图模式

MindSpore支持动态图和静态图两种模式。在静态图模式下，程序会先编译网络结构，后进行计算。编译器能利用图优化等技术对执行图进行更大程度的优化，从而获得更好的执行性能。而在动态图模式下，程序按照代码的编写顺序执行，在执行正向过程中根据反向传播的原理，动态生成反向执行图。这种模式下，编译器将神经网络中的各个算子逐一下发执行，方便用户编写和调试神经网络模型。详情参考[这里](https://www.mindspore.cn/docs/zh-CN/r2.0/design/dynamic_graph_and_static_graph.html)。

MindSpore通过一行简单的代码就能实现静态图和动态图的切换 （由于静态图的语法受限，可以先尝试用动态图模式调试，再转换成静态图模式）：

In [8]:
import mindspore as ms
# 静态图模式
# ms.set_context(mode=ms.GRAPH_MODE, device_target="CPU")
# 动态图模式
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

有了这行代码，接下来我们运行的模型计算将会在对应的静态图或者动态图模式下进行。

## 定义数据集

目前MindCV支持直接下载的数据集有`MNIST`, `CIFAR10`和`CIFAR100`。可以通过下面的代码将`CIFAR10`数据集下载到指定的文件夹：


In [9]:
import os
from mindcv.data import create_dataset, create_transforms, create_loader

cifar10_dir = './cifar-10-batches-bin'  # your dataset path
num_classes = 10  # num of classes
num_workers = 8  # num of parallel workers

# create dataset
dataset_train = create_dataset(
    name='cifar10', root=cifar10_dir, split='train', shuffle=True, num_parallel_workers=num_workers, download=True
)
dataset_test = create_dataset(
    name='cifar10', root=cifar10_dir, split='test', shuffle=False, num_parallel_workers=num_workers, download=True
)
print(f"train set samples: {len(dataset_train)}; test set samples: {len(dataset_test)}")

train set samples: 50000; test set samples: 10000


## 定义数据处理函数
接下来，我们为training set 和 test set创建data transformations。通常training set数据会经过一些数据增强，例如随机裁剪，随机翻转。例如MindCV中设置的CIFAR10 data transformation 函数：


In [10]:
from mindspore.dataset import vision
def transforms_cifar(resize=224, is_training=True):
    """Transform operation list when training or evaluating on cifar."""
    trans = []
    if is_training:
        trans += [
            vision.RandomCrop((32, 32), (4, 4, 4, 4)),
            vision.RandomHorizontalFlip(prob=0.5),
        ]

    trans += [
        vision.Resize(resize),
        vision.Rescale(1.0 / 255.0, 0.0),
        vision.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]),
        vision.HWC2CHW(),
    ]

    return trans


另外，`transforms_cifar`这个函数也可以通过下面的方式得到：

In [11]:
trans_train = create_transforms(dataset_name='cifar10', image_resize=224, is_training=True)
trans_test = create_transforms(dataset_name='cifar10', image_resize=224, is_training=False)


## 定义数据加载器

我们在数据集加载器将对应的dataset 作为参数输入，并设置batch size, num_workers等参数。

In [12]:
batch_size = 64
num_classes = 10
num_workers = 8
loader_train = create_loader(dataset=dataset_train,
                             batch_size=batch_size,
                             is_training=True,
                             num_classes=num_classes,
                             transform=trans_train,
                             num_parallel_workers=num_workers)

num_batches_train = loader_train.get_dataset_size()

loader_test = create_loader(dataset=dataset_test,
                             batch_size=batch_size,
                             is_training=False,
                             num_classes=num_classes,
                             transform=trans_test,
                             num_parallel_workers=num_workers)

num_batches_test = loader_test.get_dataset_size()
print(f"train batches: {num_batches_train} test batches: {num_batches_test}")

train batches: 782 test batches: 157


## 定义模型

MindCV 支持许多模型，例如ResNet, ViT等。用户可以通过`mindcv.list_models`查询支持的模型:

```
import mindcv
# 列出所有MindCV支持的模型
>>> mindcv.list_models("*")

# 列出所有MindCV支持的预训练模型
>>> mindcv.list_models("*", pretrained=True)

# 列出所有MindCV支持的预训练ViT模型
>>> mindcv.list_models("vit*", pretrained=True)
>>> ['vit_b_32_224', 'vit_l_16_224', 'vit_l_32_224']
```

以`vit_b_32_224`为例，创建一个`vit_b_32_224`模型只需要一行代码：


In [14]:
import mindcv
network = mindcv.create_model('vit_b_32_224',  num_classes=num_classes, pretrained=True)

[WARNING] ME(4060:139963143877568,MainProcess):2023-06-01-09:32:43.428.518 [mindspore/nn/layer/basic.py:167] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(4060:139963143877568,MainProcess):2023-06-01-09:32:43.439.070 [mindspore/nn/layer/basic.py:167] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(4060:139963143877568,MainProcess):2023-06-01-09:32:43.442.677 [mindspore/nn/layer/basic.py:167] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(4060:139963143877568,MainProcess):2023-06-01-09:32:43.459.629 [mindspore/nn/layer/basic.py:167] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(4060:139963143877568,MainProcess):2023-06-01-09:32:43.472.977 [mindspore/nn/layer/basic.py:167] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(4060:139963143877568,M

这行代码会在`~/.mindspore/models/`路径下下载一个在`ImageNet-1K`预训练好的模型权重。 在下一次创建该模型时，程序会直接载入下载好的权重。
由于`ImageNet-1K`预训练模型的`num_classes`数量与`CIFAR10`数据集的`num_classes`不相等，所以会出现一条`warning message`，提示`classifier`的权重并没有载入。这并不影响我们在`CIFAR10`数据集继续`finetun`e。您也可以选择将`pretrained`参数设为`False`, 这样模型权重会采用随机的初始化值。

## 定义损失函数

使用`create_loss`定义损失函数。 目前`create_loss`支持的损失函数有`CE` (cross entropy) 和`BCE` (binary cross entropy) 。未来还会继续支持更多常用的损失函数。
现在我们使用cross entropy作为损失函数。


In [15]:
from mindcv.loss import create_loss

loss = create_loss(name='CE')


## 定义学习率的调整策略
使用`create_scheduler`定义学习率的调整策略。`create_scheduler`支持包括`constant`, `step_decay`, `cosine_decay`在内的多种策略，详情请见[这里](https://mindcv.readthedocs.io/en/latest/api/mindcv.scheduler.html#mindcv.scheduler.create_scheduler)。在本例子中，我们使用固定的学习率$0.0001$。


In [18]:
from mindcv.scheduler import create_scheduler
# learning rate scheduler
lr_scheduler = create_scheduler(steps_per_epoch=num_batches_train,
                                scheduler='constant',
                                lr=0.0001)

## 定义优化器

使用`create_optimizer`定义损失函数。 `create_optimizer`支持各种主流的优化器，例如`adam`, `adamw`, `sgd`等。优化器的选择可以通过`opt`这个参数传递。另外`create_optimizer` 的第一个参数`params`既可以是一个由`Parameter`组成的列表，例如`network.trainable_params()`,也可以是一个字典。例如：
```
from mindspore import nn
from mindcv.optim import create_optimizer
net = Net()

# Convolutional parameter
conv_params = list(filter(lambda x: 'conv' in x.name, net.trainable_params()))
# Non-convolutional parameter
no_conv_params = list(filter(lambda x: 'conv' not in x.name, net.trainable_params()))

# Fixed learning rate
fix_lr = 0.01

# Computation of Learning Rate Based on Polynomial Decay Function
polynomial_decay_lr = nn.PolynomialDecayLR(learning_rate=0.1,      # Initial learning rate
                                           end_learning_rate=0.01, # Final the learning rate
                                           decay_steps=4,          #Number of decay steps
                                           power=0.5)              # Polynomial power

# The convolutional parameter uses a fixed learning rate of 0.001, and the weight decay is 0.01.
# The non-convolutional parameter uses a dynamic learning rate, and the weight decay is 0.0.
group_params = [{'params': conv_params, 'weight_decay': 0.01, 'lr': fix_lr},
                {'params': no_conv_params, 'lr': polynomial_decay_lr}]
optim = create_optimizer(group_params, "adam", lr=0.1, momentum=0.9, weight_decay=0.0)

```
通过字典可以针对模型的不同部分设置不同学习率等。


在本例子中，我们选择`adam`优化器对模型全部的trainable权重进行更新。


In [19]:
from mindcv.optim import create_optimizer

# create optimizer
opt = create_optimizer(network.trainable_params(), opt='adam', lr=lr_scheduler)


## 训练

[mindspore.Model](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.Model.html)提供了非常丰富的接口。用户可以传入与训练相关的各种参数，例如损失函数，评价函数， 混合精度。

我们将前面定义好的损失函数，学习率策略，优化器和模型作为参数传递给`Model`：


In [20]:
from mindspore import Model

# Encapsulates examples that can be trained or inferred
model = Model(network, loss_fn=loss, optimizer=opt, metrics={'accuracy'})


[mindspore.Model.train](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.Model.html#mindspore.Model.train) 能够开启训练过程，并且根据传入函数的参数，设置训练所需要的轮次(epochs），训练数据集，回调对象（callbacks, 在训练中途进行loss printing, checkpoint saving 等操作）。另外，要理解`dataset_sink_mode`参数可以参考这里的[资料](https://www.mindspore.cn/tutorials/experts/zh-CN/master/optimize/execution_opt.html)。简单理解就是开启模型计算和数据载入的并行模式。在`PYNATIVE`模式下，`dataset_sink_mode`默认是关闭的。关闭`dataset_sink_mode`有助于调试， 开启`dataset_sink_mode`通常能够获得更快的训练速度。

回调对象可以丰富我们对训练过程的监控。 这里我们设置了每`num_batches_train//5`个steps就输出一次损失函数的值。

In [22]:
from mindspore import LossMonitor, TimeMonitor, CheckpointConfig, ModelCheckpoint

# Set the callback function for saving network parameters during training.
ckpt_save_dir = './ckpt'
ckpt_config = CheckpointConfig(save_checkpoint_steps=num_batches_train)
ckpt_cb = ModelCheckpoint(prefix='vit-cifar10',
                          directory=ckpt_save_dir,
                          config=ckpt_config)
loss_monitor = LossMonitor(num_batches_train//5)
time_monitor = TimeMonitor(num_batches_train//5)
num_epochs = 5
model.train(num_epochs, loader_train, callbacks=[loss_monitor,time_monitor, ckpt_cb], dataset_sink_mode=False)


epoch: 1 step: 156, loss is 2.1286168098449707
epoch: 1 step: 312, loss is 1.1996170282363892
epoch: 1 step: 468, loss is 0.471361368894577
epoch: 1 step: 624, loss is 0.2324763387441635
epoch: 1 step: 780, loss is 0.20593130588531494
Train epoch time: 1864830.944 ms, per step time: 2384.694 ms
epoch: 2 step: 154, loss is 0.14241477847099304
epoch: 2 step: 310, loss is 0.13663174211978912
epoch: 2 step: 466, loss is 0.16266559064388275
epoch: 2 step: 622, loss is 0.12892954051494598
epoch: 2 step: 778, loss is 0.11221715807914734
Train epoch time: 1768254.883 ms, per step time: 2261.196 ms
epoch: 3 step: 152, loss is 0.19256466627120972
epoch: 3 step: 308, loss is 0.23028206825256348
epoch: 3 step: 464, loss is 0.1350468248128891
epoch: 3 step: 620, loss is 0.026827018707990646
epoch: 3 step: 776, loss is 0.1309332251548767
Train epoch time: 1788569.746 ms, per step time: 2287.174 ms
epoch: 4 step: 150, loss is 0.06645049899816513
epoch: 4 step: 306, loss is 0.12062713503837585
epoch: 

## 测试

训练完成后，我们对模型的准确度进行测试，结果如下：


In [23]:
acc = model.eval(loader_test, dataset_sink_mode=False)
print(acc)

{'accuracy': 0.9719}
